In [ ]:
# Here we are using the same Skeleton of chatbot and improving it by adding loops and conditions to store the chat history.
from langgraph.graph import StateGraph,START,END
from langchain_openai import ChatOpenAI
from typing import TypedDict, Literal
import operator
from typing import List,Annotated
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [ ]:
load_dotenv()

llm=ChatOpenAI()

In [ ]:
class JokeState(TypedDict):
    topic:str
    joke:str
    explanation:str


In [ ]:
def generate_joke(state:JokeState):
    prompt=f"generate a joke for the topic :{state['topic']}"
    result=llm.invoke(prompt).content
    return {'joke':result}

def generate_explanation(state:JokeState):
    prompt=f"generate a explanation for the joke :{state['joke']}"
    result=llm.invoke(prompt).content
    return {'explanation': result}


In [ ]:
graph =StateGraph(JokeState)
graph.add_node('generate_joke',generate_joke)
graph.add_node('generate_explanation',generate_explanation)

graph.add_edge(START,'generate_joke')
graph.add_edge('generate_joke','generate_explanation')
graph.add_edge('generate_explanation',END)

#now build the checkpointer object for InMemorySaver class 
checkpointer=InMemorySaver()


workflow=graph.compile(checkpointer=checkpointer)


#Now create a threadId for this conversation
config={'configurable':{'thread_id':'1'}} 
workflow.invoke({'topic':"pizza"},config=config)

{'topic': 'pizza',
 'joke': "Why did the pizza go to the party alone? Because it didn't want to be too cheesy!",
 'explanation': 'This joke plays on the double meaning of the word "cheesy." In one sense, cheesy can refer to having characteristics of cheese, which is often associated with pizza. In another sense, cheesy can also mean corny or overly sentimental. So, in this joke, the pizza decided to go to the party alone because it didn\'t want to be too cheesy in the sense of being overly corny or sentimental.'}

In [ ]:
workflow.get_state(config)

StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the party alone? Because it didn't want to be too cheesy!", 'explanation': 'This joke plays on the double meaning of the word "cheesy." In one sense, cheesy can refer to having characteristics of cheese, which is often associated with pizza. In another sense, cheesy can also mean corny or overly sentimental. So, in this joke, the pizza decided to go to the party alone because it didn\'t want to be too cheesy in the sense of being overly corny or sentimental.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e1658-26e2-62fa-8002-ab571527ea80'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-12-25T07:44:14.468786+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e1658-1947-6fab-8001-f62204c74c5d'}}, tasks=(), interrupts=())

In [ ]:
list(workflow.get_state_history(config)) # this will give all the intermediate state history of the graph

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the party alone? Because it didn't want to be too cheesy!", 'explanation': 'This joke plays on the double meaning of the word "cheesy." In one sense, cheesy can refer to having characteristics of cheese, which is often associated with pizza. In another sense, cheesy can also mean corny or overly sentimental. So, in this joke, the pizza decided to go to the party alone because it didn\'t want to be too cheesy in the sense of being overly corny or sentimental.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e1658-26e2-62fa-8002-ab571527ea80'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-12-25T07:44:14.468786+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e1658-1947-6fab-8001-f62204c74c5d'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go

In [ ]:
workflow.get_state(config)